In [11]:
class Board():
    cols = 10
    rows = 10
    blank = "_"
    hit = "*"
    ship_types = {'frigate': 3, 'scout': 2, 'battle': 4}

    def __init__(self):
        self.board = [[self.blank] * self.rows for _ in range(self.cols)]
        self.ships = []

    @staticmethod
    def scan_board(horiz, vertiz):
        for row_ in range(*horiz):
            for col_ in range(*vertiz):
                yield row_, col_

    @staticmethod
    def set_scan_area(x, y, length, horizontal = True):
        if horizontal:
            if y + length > Board.cols or y < 0:
                raise IndexError('Off of board')
            return (x, x + 1), (y, y + length)
        else:
            if x + length > Board.rows or x < 0:
                raise IndexError('Off of board')
            return (x, x + length), (y, y + 1)
        
    def full_of_symbol(self, horiz, vertiz, symbol):
        ''' check if region is full of symbol '''
        for row, col in Board.scan_board(horiz, vertiz):
            if self.board[row][col] != symbol:
                return False
        return True        
    
    
    def is_empty(self, horiz, vertiz):
        ''' check if region of board is empty '''
        return self.full_of_symbol(horiz, vertiz, Board.blank)

    
    def is_all_hit(self, horiz, vertiz):
        ''' check if region of board is full of hits '''
        return self.full_of_symbol(horiz, vertiz, Board.hit)
    
    def is_sunk(self, x, y):
        ''' if a hit sinks boat - remove from board and return True '''
        for index, ship in enumerate(self.ships):
            if x in range(*ship[0]) and y in range(*ship[1]):
                if self.is_all_hit(ship[0], ship[1]):
                    self.set_symbol(ship[0], ship[1], Board.blank)
                    self.ships.pop(index)
                    return True
        return False

    def set_symbol(self, horiz, vertiz, symbol):
        for row, col in Board.scan_board(horiz, vertiz):
            self.board[row][col] = symbol
    
    def add_ship(self, ship_name, horiz, vertiz, symbol):
        ''' place ship on board and add to list of ships '''
        self.ships.append((horiz, vertiz, symbol, ship_name))
        self.set_symbol(horiz, vertiz, symbol)
    
    def hit_ship(self, x, y):
        ''' see if coordinates hit a ship '''
        print(f'checking {x}{y} which is {self.board[x][y]}')
        if 0 <= x <= self.rows and 0 <= y <= self.cols:
            if not self.board[x][y] in [Board.blank, Board.hit]:
                self.board[x][y] = Board.hit
                return True
        else:
            return False
    
    def firing_results(self, x, y):
        a_hit = self.hit_ship(x, y)
        a_sinking = False
        if a_hit:
            a_sinking = self.is_sunk(x, y)                    
        return a_hit, a_sinking
    
    def all_gone(self):
        ''' check if any ships left '''
        return not len(self.ships)

    @staticmethod
    def get_coords(msg="Enter coordinates: x y: "):
        while True:
            try:
                coords = [int(num) for num in input(msg).strip().split()]
            except ValueError as e:
                pass
            else:
                if coords and len(coords) == 2 and \
                0 <= coords[0] <= Board.rows and \
                0 <= coords[1] <= Board.cols:
                    break
            print(f"Two positive integer numbers seperated by a space, each within range"
                  f" {Board.rows} x {Board.cols} are requied.")
        return coords
                    
            
    
    def set_ship(self, ship_type, x, y, length = 3, symbol = "*", horizontal = True):
        try:
            horiz, vertiz = Board.set_scan_area(x, y, length, horizontal)
        except IndexError as e:
            print(f'row {x},  col: {y}, length: {length} is {e}. IGNORED.\n')
        else:
            if self.is_empty(horiz, vertiz):
                self.add_ship(ship_type, horiz, vertiz, symbol)
                return True
            else:
                print(f'row {x},  col: {y}, length: {length} is not empty. IGNORED.\n') 
        return False

    def __repr__(self):
        header = '  ' + ''.join([f'{n % 10:>2d}' for n in range(self.cols)]) + '\n'
        rows = [f'{index % 10:<2d}' + ' '.join(row) for index, row in enumerate(self.board)]
        return header + '\n'.join(rows)


    def position_ships(self, *ships):
        for index, ship in enumerate(ships):
            print('\n', self, '\n')
            DEBUG = True
            DEBUG_data = [(0,0), (1,1), (2,2), (3,3)]
            while True:
                print(f"You have a {ship} of length {self.ship_types[ship]} to position.")
                if DEBUG:
                    coords = list(DEBUG_data.pop())
                    orientation = True
                else:
                    coords = self.get_coords()
                    orientation = input('Do you want it horizontal? ').strip().lower() \
                            in ['yes', 'y', 'ok', 'h', 'horiz', 'horizontal']
                if self.set_ship(ship, coords[0], coords[1], self.ship_types[ship],
                                 symbol = f'{index:1d}', horizontal = orientation):
                    break
        
    def move(self):
        coords = self.get_coords()
        hit, sunk = self.firing_results(*coords)
        if hit:
            print('A Hit!')
        if sunk:
            print('A Sinking!')
        return self.all_gone()        

def players_fighting(players):
    in_play = 0
    for player in players:
        if player.ships:
            in_play += 1
    return in_play

HORZ = True
VERT = False

players = []
number_of_players = 2
for _ in range(number_of_players):
    players.append(Board())

for index, player in enumerate(players, start = 1):
    print(f'\n\nPlayer {index} - you need to position your ships\n\n')
    player.position_ships('frigate', 'scout', 'battle', 'frigate')
    print(f'Player {index}: \n\n', player)

while players_fighting(players) > 1:
    for index, player in enumerate(players, start = 1):
        attacker = index + 1 if index < number_of_players else 1
        print(f'Player number {attacker} to attack Player number {index}.')
        print('')
        print('DEBUG VIEW: \n', player)
        if player.move():
            print(f'Player {index} out of the game.')



Player 1 - you need to position your ships



    0 1 2 3 4 5 6 7 8 9
0 _ _ _ _ _ _ _ _ _ _
1 _ _ _ _ _ _ _ _ _ _
2 _ _ _ _ _ _ _ _ _ _
3 _ _ _ _ _ _ _ _ _ _
4 _ _ _ _ _ _ _ _ _ _
5 _ _ _ _ _ _ _ _ _ _
6 _ _ _ _ _ _ _ _ _ _
7 _ _ _ _ _ _ _ _ _ _
8 _ _ _ _ _ _ _ _ _ _
9 _ _ _ _ _ _ _ _ _ _ 

You have a frigate of length 3 to position.

    0 1 2 3 4 5 6 7 8 9
0 _ _ _ _ _ _ _ _ _ _
1 _ _ _ _ _ _ _ _ _ _
2 _ _ _ _ _ _ _ _ _ _
3 _ _ _ 0 0 0 _ _ _ _
4 _ _ _ _ _ _ _ _ _ _
5 _ _ _ _ _ _ _ _ _ _
6 _ _ _ _ _ _ _ _ _ _
7 _ _ _ _ _ _ _ _ _ _
8 _ _ _ _ _ _ _ _ _ _
9 _ _ _ _ _ _ _ _ _ _ 

You have a scout of length 2 to position.
row 3,  col: 3, length: 2 is not empty. IGNORED.

You have a scout of length 2 to position.

    0 1 2 3 4 5 6 7 8 9
0 _ _ _ _ _ _ _ _ _ _
1 _ _ _ _ _ _ _ _ _ _
2 _ _ 1 1 _ _ _ _ _ _
3 _ _ _ 0 0 0 _ _ _ _
4 _ _ _ _ _ _ _ _ _ _
5 _ _ _ _ _ _ _ _ _ _
6 _ _ _ _ _ _ _ _ _ _
7 _ _ _ _ _ _ _ _ _ _
8 _ _ _ _ _ _ _ _ _ _
9 _ _ _ _ _ _ _ _ _ _ 

You have a battle of length 4 to p

KeyboardInterrupt: 

In [ ]:
players[0]